In [243]:
api_key = "your_api_key"

In [244]:
import requests
import json
from pprint import pprint
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [245]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"
            "https://www.googleapis.com/auth/youtube.force-ssl"]

youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey = api_key)

In [101]:
coordsUAE = "23.424076, 53.847818"
radiusUAE = "100km"

coordsQatar = "25.286106, 51.534817"
radiusQatar = "80km"

coordsKSA = "24.774265, 46.738611"
radiusKSA = "200km"

In [102]:
videoIds = []

In [248]:
data = {}
# data['results'] = []

Function to get results from the api with provided coordinates (Country based)

In [105]:
def vidSearchCord(keyword, coords, radius, pagetoken):
    request = youtube.search().list(
        part="id",
        location=coords,
        locationRadius=radius,
        pageToken = pagetoken,
        maxResults = 30,
        q=keyword,
        type="video")
    
    return request.execute()

Function to get results from the api with provided keywords

In [106]:
def vidSearchKey(keywords, pagetoken):
    request = youtube.search().list(
        part="id",
        pageToken = pagetoken,
        maxResults = 50,
        q=keywords,
        type="video")
    
    return request.execute()

Helper function to save all videos initially in a list

In [107]:
def getVids(result):
    for item in result['items']:
        id = item['id']['videoId']
        if id not in videoIds:
            videoIds.append(id)
        
    return

Helper function to retrieve the number of comment, number of favorite, time of publication and the title of the video

In [229]:
def getVidInfo(videoId):
    request = youtube.videos().list(
        part="snippet, statistics",
        id=videoId).execute()
    
    stat = request['items'][0]['statistics']
    publishedTime = request['items'][0]['snippet']['publishedAt']
    title = request['items'][0]['snippet']['title']
    
    if 'commentCount' in stat:
        comCount = stat['commentCount']
    else:
        comCount = 0
        
    if 'favoriteCount' in stat:
        favCount = stat['favoriteCount']
    else:
        favCount = 0
        
    return [int(comCount), favCount, publishedTime, title]

Function to get comments from the api 

In [230]:
def commentThread(videoId, pagetoken):
    results = youtube.commentThreads().list(
        part="snippet",
        maxResults=100,
        pageToken = pagetoken,
        videoId=videoId)
    
    return results.execute()

Helper funtion to structure the information's of every comment

In [231]:
def loadComments(id, found):
#     data[id]['comments'] = []
    for item in found['items']:
        commentId = item['id']
        authorName = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
        comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
        publishedTime = item['snippet']['topLevelComment']['snippet']['publishedAt']
        totalLike = item['snippet']['topLevelComment']['snippet']['likeCount']
        totalReplies = item['snippet']['totalReplyCount']
        data[id]['comments'].append({
            'commentId':commentId,
            'commentAuthorName':authorName,
            'commentText':comment,
            'commentMadeOn':publishedTime,
            'totalLikeCount':totalLike,
            'totalRepliesCount':totalReplies
        })
#         if totalReplies > 0:
#             for ind in range(totalReplies):
#                 data['results'][index]['comments'][ind]['replies'] = []
#                 for reply in item['replies']['comments']:
#                     replyId = reply['id']
#                     rAuthorName = reply['snippet']['authorDisplayName']
#                     rComment = reply['snippet']['textOriginal']
#                     data['results'][index]['comments'][ind]['replies'].append({
#                         'replyId':replyId,
#                         'replyAuthor':rAuthorName,
#                         'replyText':rComment
#                     })
    return

Helper function to retrieve all comments from a video

In [232]:
def getAllVidComment(id):
#     data[id] = {}
    vidInfo = getVidInfo(id)
    comCount = vidInfo[0]
    favCount = vidInfo[1]
    publishedTime = vidInfo[2]
    title = vidInfo[3]
    data[id] = {
        'videoTitle':title,
        'videoPublishedOn':publishedTime,
        'favoriteCount':favCount,
        'totalComment':comCount,
        'comments':[]
    }
    if comCount>0:
        comments = commentThread(id, "")
        loadComments(id, comments)
        if 'nextPageToken' in comments:
            nextPage = comments['nextPageToken']
        
            while nextPage:
                comments = commentThread(id, nextPage)
                loadComments(id, comments)
                if 'nextPageToken' in comments:
                    nextPage = comments['nextPageToken']
                else:
                    return

Search by Key word

In [112]:
def getResFromKeyword():
    keyword = ["Talabat Qatar", "Talabat Middle East", "Carriage Qatar", "Uber Eats Middle East", "Zomato Middle East",
               "Careem Now Food Delivery Middle East", "Food Delivery Middle Eas"]
    for word in keyword:
        res = vidSearchKey(word, "")
        getVids(res)
    
    return

Search by location

In [113]:
def getResFromLocation():
    key = ["Talabat", "Careem Now Food Delivery", "Uber Eats", "Zomato", "Food Delivery"]
    for wordQ in key:
        resQ = vidSearchCord(wordQ, coordsQatar, radiusQatar, "")
        getVids(resQ)
    for wordU in key:
        resU = vidSearchCord(wordU, coordsUAE, radiusUAE, "")
        getVids(resU)
    for wordK in key:
        resK = vidSearchCord(wordK, coordsKSA, radiusKSA, "")
        getVids(resK)
    return

In [114]:
getResFromKeyword()
getResFromLocation()

In [250]:
for index in range(len(videoIds)):
    getAllVidComment(videoIds[index])

Save into json file 

In [252]:
with open('dataset.json', 'w') as outfile:
    json.dump(data, outfile, indent=4, sort_keys=True)

THE END LOL :)